## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-18-48-58 +0000,nypost,10 illegal migrant kids found at California po...,https://nypost.com/2025/07/11/us-news/10-illeg...
1,2025-07-11-18-46-46 +0000,nypost,Numbskull snaps pics of NYC sunset while subwa...,https://nypost.com/2025/07/11/us-news/numbskul...
2,2025-07-11-18-35-18 +0000,bbc,Draw puts Mali on brink of Wafcon quarter-finals,https://www.bbc.com/sport/football/articles/c7...
3,2025-07-11-18-22-44 +0000,nyt,Republicans Blame Canada for Wildfire Smoke ‘S...,https://www.nytimes.com/2025/07/11/world/canad...
4,2025-07-11-18-21-04 +0000,nyt,Trump Says NATO Countries Will Buy Weapons to ...,https://www.nytimes.com/2025/07/11/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,11
57,tariff,5
28,wafcon,4
351,brazil,4
168,off,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
18,2025-07-11-15-59-15 +0000,nypost,Head Start education program will be cut off f...,https://nypost.com/2025/07/11/us-news/head-sta...,29
8,2025-07-11-17-30-02 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,27
22,2025-07-11-15-18-54 +0000,nypost,Trump teases ‘major statement’ on Russia after...,https://nypost.com/2025/07/11/us-news/trump-te...,27
53,2025-07-10-20-10-28 +0000,nyt,"Targeting Brazil, Trump Tests Legal Limit of H...",https://www.nytimes.com/2025/07/10/us/politics...,26
47,2025-07-11-01-45-42 +0000,nyt,Trump Tariff Letters: See the Latest Rates and...,https://www.nytimes.com/2025/07/07/business/tr...,26


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
18,29,2025-07-11-15-59-15 +0000,nypost,Head Start education program will be cut off f...,https://nypost.com/2025/07/11/us-news/head-sta...
40,21,2025-07-11-12-29-49 +0000,nypost,Tom Homan rips violent clashes at California p...,https://nypost.com/2025/07/11/us-news/tom-homa...
29,17,2025-07-11-14-04-24 +0000,nypost,Sen. Bill Cassidy’s campaign touts record-sett...,https://nypost.com/2025/07/11/us-news/sen-bill...
36,17,2025-07-11-13-26-18 +0000,nypost,"Brave teen girl, 17, died saving her entire fa...",https://nypost.com/2025/07/11/us-news/teenage-...
8,16,2025-07-11-17-30-02 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
20,14,2025-07-11-15-45-10 +0000,nypost,Hit-and-run BMW driver kills two pedestrians o...,https://nypost.com/2025/07/11/us-news/hit-and-...
49,14,2025-07-10-21-44-33 +0000,bbc,Nigeria win to book spot in Wafcon quarter-finals,https://www.bbc.com/sport/football/articles/cg...
11,13,2025-07-11-16-57-34 +0000,bbc,Mother mourns 'beautiful' 12-year-old shot whi...,https://www.bbc.com/news/articles/cd6gzx85gdqo
12,13,2025-07-11-16-33-10 +0000,nypost,Camp Mystic hero killed saving girls spent yea...,https://nypost.com/2025/07/11/us-news/hero-tex...
16,13,2025-07-11-16-01-08 +0000,nypost,Idaho murder victim’s mom finds unexpected pea...,https://nypost.com/2025/07/11/us-news/idaho-mu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
